In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# Importa la biblioteca PyTorch, que se utiliza para construir y entrenar redes neuronales.
import torch
#Importa la biblioteca NumPy, que se utiliza para operaciones numéricas eficientes en Python.
import numpy as np
#Importa la biblioteca Pandas, que se utiliza para trabajar con estructuras de datos tabulares, como DataFrames.
import pandas as pd

In [67]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/machine learning/Datasets/pizza_v1.csv')
num_filas, num_columnas = data.shape

keys = data.keys()
print(keys)
print(f'Número de filas: {num_filas}\n')
print(f'Número de columnas: {num_columnas}\n')
#num_labels = data['posttest'].nunique()
#print("Número de etiquetas únicas para y:", num_labels)

Index(['company', 'price_rupiah', 'diameter', 'topping', 'variant', 'size',
       'extra_sauce', 'extra_cheese'],
      dtype='object')
Número de filas: 129

Número de columnas: 8



In [68]:
# Elimina las columnas "company" y "variant"
data = data.drop(["company", "variant", "topping", "size"], axis=1)
columna_categorica = 'price_rupiah'
columna_numerica = 'price'

#Eliminar simbolos
columnas_eliminar = ['price_rupiah']
simbolo = 'Rp'
for columna in columnas_eliminar:
    data[columna] = data[columna].str.replace(simbolo, '', regex=False)
    data[columna_categorica] = data[columna_categorica].str.replace(',', '', regex=False)

# Crea una nueva columna numérica y copia los valores de la columna categórica
data[columna_numerica] = pd.to_numeric(data[columna_categorica], errors='coerce')

# Elimina la columna categórica original
data.drop(columna_categorica, axis=1, inplace=True)
print(data.keys())
from sklearn.preprocessing import LabelEncoder
#variables categoricas
columnas_categoricas = data.select_dtypes(include=['object']).columns
#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])


Index(['diameter', 'extra_sauce', 'extra_cheese', 'price'], dtype='object')


In [69]:
X, Y = data.iloc[:, :3], data.iloc[:, 3]
print(X.shape)
print(Y.shape)
print(X,Y)

(129, 3)
(129,)
     diameter  extra_sauce  extra_cheese
0        22.0            1             1
1        20.0            1             1
2        16.0            1             1
3        14.0            1             0
4        18.0            1             0
..        ...          ...           ...
124       8.5            1             1
125      12.0            1             1
126      14.0            1             1
127       8.5            1             0
128      12.0            0             0

[129 rows x 3 columns] 0      235000
1      198000
2      120000
3      155000
4      248000
        ...  
124     39000
125     72000
126     99000
127     44000
128     78000
Name: price, Length: 129, dtype: int64


In [70]:
from sklearn.model_selection import train_test_split
# Dividir los datos en entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Asegurarse de que las etiquetas sean de tipo entero
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
y_train = y_train.astype(np.float)
y_test = y_test.astype(np.float)

#print(X_test.info())
#print(X_train.info())

<ipython-input-70-92c826b0c764>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = X_train.astype(np.float)
<ipython-input-70-92c826b0c764>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = X_test.astype(np.float)
<ipython-input-70-92c826b0c764>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float

In [71]:
"""print("X_train:\n",X_train)
print("y_train:\n",y_train)
print("X_test:\n",X_test)
print("y_test:\n",y_test)"""

'print("X_train:\n",X_train)\nprint("y_train:\n",y_train)\nprint("X_test:\n",X_test)\nprint("y_test:\n",y_test)'

###Modelo

In [72]:
import torch.nn as nn
import torch.optim as optim
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Definir un modelo de regresión lineal simple
class ComplexRegression(nn.Module):
    def __init__(self, input_size):
        super(ComplexRegression, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Capa oculta con 128 neuronas
        self.relu = nn.ReLU ()  # Función de activación ReLU
        self.fc2 = nn.Linear(128, 1)  # Capa de salida lineal

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# Crear una instancia del modelo
input_size = X_train.shape[1]
model = ComplexRegression(input_size)

# Definir la función de pérdida (loss)
#criterion = nn.MSELoss()  # Error cuadrático medio (Mean Squared Error, MSE)
#criterion = nn.MSELoss()  # Error cuadrático medio logarítmico (MSLE)
criterion = nn.SmoothL1Loss()  # Error de Huber
#criterion = nn.L1Loss()  # Error Absoluto Medio (MAE)

# Definir el optimizador
optimizer = optim.SGD(model.parameters(), lr=0.001)  # Descenso por el gradiente estocástico (SGD)

# Entrenar el modelo
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))  # Asegurarse de que las dimensiones coincidan

    # Backward pass y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluar el modelo en datos de prueba
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test.view(-1, 1))

print(f'Loss en los datos de prueba: {test_loss.item():.4f}')


Epoch [10/100], Loss: 87486.5469
Epoch [20/100], Loss: 87473.8828
Epoch [30/100], Loss: 87459.8438
Epoch [40/100], Loss: 87443.4531
Epoch [50/100], Loss: 87423.6094
Epoch [60/100], Loss: 87398.9531
Epoch [70/100], Loss: 87367.7969
Epoch [80/100], Loss: 87328.0078
Epoch [90/100], Loss: 87276.9062
Epoch [100/100], Loss: 87211.0000
Loss en los datos de prueba: 85501.4141


In [73]:
#Predicciones en los datos de entrenamiento:
with torch.no_grad():
    y_pred_train = model(X_train)
    test_loss = criterion(y_pred_train, y_train.view(-1, 1))
print(f'Loss en los datos de prueba: {test_loss.item():.4f}')
#Predicciones en los datos de prueba:
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = criterion(y_pred_test, y_test.view(-1, 1))
print(f'Loss en los datos de prueba: {test_loss.item():.4f}')

Loss en los datos de prueba: 87203.4141
Loss en los datos de prueba: 85501.4141


In [74]:
import numpy as np

# Calcular el MAPE
y_true = y_test.numpy()
y_predi = y_pred.numpy()
mape = np.mean(np.abs((y_true - y_predi) / y_true)) * 100

print(f'Error Porcentual Absoluto Medio (MAPE): {mape:.2f}%')



Error Porcentual Absoluto Medio (MAPE): 99.59%
